In [3]:
%load_ext autoreload
%autoreload 2

import sys
import os
from enum import Enum
import numpy as np
import torch

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.image import imread, imsave
import h5py

def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)


def plotData(dataY, rangeY=None, dataYR=None, rangeYR=None,
             dataX=None, rangeX=None, rangeP=None,
             figsize=(16,8), saveTo=None, show=True):

    if type(dataY) is np.ndarray :
        plotData((dataY,), rangeY=rangeY, dataYR=dataYR, rangeYR=rangeYR,
             dataX=dataX, rangeX=rangeX, rangeP=rangeP,
             figsize=figsize, saveTo=saveTo, show=show)
        return
    if type(dataYR) is np.ndarray :
        plotData(dataY, rangeY=rangeY, dataYR=(dataYR,), rangeYR=rangeYR,
             dataX=dataX, rangeX=rangeX, rangeP=rangeP,
             figsize=figsize, saveTo=saveTo, show=show)
        return
    if type(dataY) is not tuple :
        eprint(f"Unknown data type to plot: {type(dataY)}.")
        return
    if type(dataYR) is not tuple and dataYR is not None:
        eprint(f"Unknown data type to plot: {type(dataYR)}.")
        return

    last = min( len(data) for data in dataY )
    if dataYR is not None:
        last = min( last,  min( len(data) for data in dataYR ) )
    if dataX is not None:
        last = min(last, len(dataX))
    if rangeP is None :
        rangeP = (0,last)
    elif type(rangeP) is int :
        rangeP = (0,rangeP) if rangeP > 0 else (-rangeP,last)
    elif type(rangeP) is tuple :
        rangeP = ( 0    if rangeP[0] is None else rangeP[0],
                   last if rangeP[1] is None else rangeP[1],)
    else :
        eprint(f"Bad data type on plotData input rangeP: {type(rangeP)}")
        raise(f"Bug in the code.")
    rangeP = np.s_[ max(0, rangeP[0]) : min(last, rangeP[1]) ]
    if dataX is None :
        dataX = np.arange(rangeP.start, rangeP.stop)

    plt.style.use('default')
    plt.style.use('dark_background')
    fig, ax1 = plt.subplots(figsize=figsize)
    ax1.xaxis.grid(True, 'both', linestyle='dotted')
    if rangeX is not None :
        ax1.set_xlim(rangeX)
    else :
        ax1.set_xlim(rangeP.start,rangeP.stop-1)

    ax1.yaxis.grid(True, 'both', linestyle='dotted')
    nofPlots = len(dataY)
    if rangeY is not None:
        ax1.set_ylim(rangeY)
    colors = [ matplotlib.colors.hsv_to_rgb((hv/nofPlots, 1, 1)) for hv in range(nofPlots) ]
    for idx , data in enumerate(dataY):
        ax1.plot(dataX, data[rangeP], linestyle='-',  color=colors[idx])

    if dataYR is not None : # right Y axis
        ax2 = ax1.twinx()
        ax2.yaxis.grid(True, 'both', linestyle='dotted')
        nofPlots = len(dataYR)
        if rangeYR is not None:
            ax2.set_ylim(rangeYR)
        colors = [ matplotlib.colors.hsv_to_rgb((hv/nofPlots, 1, 1)) for hv in range(nofPlots) ]
        for idx , data in enumerate(dataYR):
            ax2.plot(dataX, data[rangeP], linestyle='dashed',  color=colors[idx])

    if saveTo:
        fig.savefig(saveTo)
    if not show:
        plt.close(fig)





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## <font style="color:blue">Data</font>

In [ ]:



procsFolder = "procs"
gapWidth = 12
sinoLen = 4096

class StripesFromHDF :

    def __init__(self, sampleName, maskName, bgName=None, dfName=None, loadToMem=True):

        sampleHDF = sampleName.split(':')
        if len(sampleHDF) != 2 :
            raise(f"String \"{sampleName}\" does not represent an HDF5 format.")
        with h5py.File(sampleHDF[0],'r') as trgH5F:
            if  sampleHDF[1] not in trgH5F.keys():
                raise(f"No dataset '{sampleHDF[1]}' in input file {sampleHDF[0]}.")
            self.data = trgH5F[sampleHDF[1]]
            if not self.data.size() :
                raise(f"Container \"{sampleName}\" is zero size.")
            self.sh = self.data.shape
            self.fsh = self.data.shape[0,1]
            if len(self.sh) != 3 :
                raise(f"Dimensions of the container \"{sampleName}\" is not 3 ({self.sh}).")
            self.volume = None
            if loadToMem :
                self.volume = self.data[:]
                trgH5F.close()

            self.mask = imread(maskName).astype(bool)
            if self.mask.shape != self.fsh :
                raise(f"Dimensions of the mask image \"{maskName}\" ({self.mask.shape}) do not match the face "
                      f"of the container \"{sampleName}\" ({self.fsh}).")
            self.bg = imread(bgName) if bgName else None
            if self.bg and self.bg.shape != self.fsh :
                raise(f"Dimensions of the BG image \"{bgName}\" ({self.bg.shape}) do not match the face "
                      f"of the container \"{sampleName}\" ({self.fsh}).")
            self.df = imread(dfName) if self.dfName else None
            if self.df and self.df.shape != self.fsh :
                raise(f"Dimensions of the DF image \"{dfName}\" ({self.df.shape}) do not match the face "
                      f"of the container \"{sampleName}\" ({self.fsh}).")
            if self.bg :
                if self.df :
                    self.bg -= self.df
                self.mask  &=  self.bg > 0.0

            self.allIndices = []
            for yCr in range(0,self.fsh[0]) :
                for xCr in range(0,self.fsh[1]) :
                    idx = np.s_[yCr,xCr]
                    if xCr + 3*gapWidth < self.fsh[1] and np.all( self.mask[yCr,xCr:xCr+3*gapWidth] ) :
                        self.allIndices.append(idx)
                        if self.volume :
                            if self.df :
                                self.volume[*idx,:] -= self.df[idx]
                            if self.bg :
                                self.volume[*idx,:] /= self.bg[idx]


    def get_dataset(self) :
        class Sinos(torch.utils.data.Dataset) :
            def __init__(self, root):
                self.container = root
            def __len__(self):
                return len(self.container.allIndeces)
            def __getitem__(self, index):
                idx = self.container.allIndeces[index]
                xyrng=np.s_[ idx[0], idx[1]:idx[1]+3*gapWidth ]
                if self.container.volume :
                    return self.container.volume[:, *xyrng]
                else :
                    data = self.container.data[:, *xyrng]
                    if self.container.df :
                        data -= self.container.df[None,*xyrng]
                    if self.container.bg :
                        data /= self.container.bg[None,*xyrng]
                    return data
        return Sinos(self)


    def get_data_loader(self, batch_size, shuffle=None, num_workers=os.cpu_count() ) :
        return torch.utils.data.DataLoader( self.get_dataset(),
                                            batch_size=batch_size,
                                            num_workers=num_workers,
                                            shuffle = shuffle)

sinoRoot = StripesFromHDF("test.hdf:/data", "mask.tif", "bg.tif", "df.tif")

